In [ ]:
#from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
#from surprise import evaluate, print_perf
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor, KNNBaseline, SlopeOne
from surprise import accuracy
from surprise.model_selection import KFold
import io
import pandas as pd

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1) #读卡器，指定读取的列名
data = Dataset.load_from_file('./ratings.csv', reader=reader)
train_set = data.build_full_trainset()

#优化器
# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5} #reg_i是物品的正则化参数，默认为10；reg_u是用户的正则化参数，默认为15 ；n_epochs是迭代次数，默认为10
# SGD优化
#bsl_options = {'method': 'sgd','n_epochs': 5} #reg是代价函数的正则化项，默认为0.02；learning_rate是学习率，默认为0.005；n_epochs是迭代次数，默认为20

#模型
#BaselineOnly
algo = BaselineOnly(bsl_options=bsl_options)
#algo = BaselineOnly()
#NormalPredictor
#algo = NormalPredictor()
# 使用SlopeOne算法
algo = SlopeOne()

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

# 读取物品（电影）名称信息
def read_item_names():
    file_name = ('./movies.csv') 
    data = pd.read_csv('./movies.csv')
    rid_to_name = {}
    name_to_rid = {}
    for i in range(len(data['movieId'])):
        rid_to_name[data['movieId'][i]] = data['title'][i]
        name_to_rid[data['title'][i]] = data['movieId'][i]

    return rid_to_name, name_to_rid 

# 相似度计算，使用皮尔逊相似度计算法，使用ItemCF的相似度计算
sim_options = {'name': 'pearson_baseline', 'user_based': False}
# 使用KNNBaseline算法，一种CF算法
algo = KNNBaseline(sim_options=sim_options)
algo.fit(train_set)
#获得电影名称信息数据
rid_to_name, name_to_rid = read_item_names()
#获得Toy Story电影的电影ID
toy_story_raw_id = name_to_rid['Toy Story (1995)']
print(toy_story_raw_id)
#通过Toy Story电影的电影ID获取该电影的推荐内部id
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)
print(toy_story_inner_id)